In [1]:
from IPython.core.display import display, HTML
import numpy as np
import pandas as pd
import openrouteservice

In [2]:
CVRP = np.array([[  8.683448,  49.416961,   0.  ],
                 [  8.681247,  49.412388,  22.  ],
                 [  8.662789,  49.37446,    7.  ],
                 [  8.663488,  49.390504,  22.  ],
                 [  8.650365,  49.424738,   8.  ],
                 [  8.684574,  49.393819,  17.  ],
                 [  8.636446,  49.430018,   8.  ],
                 [  8.693395,  49.401669,  25.  ],
                 [  8.685109,  49.387783,  17.  ],
                 [  8.670162,  49.406377,   4.  ],
                 [  8.63245,   49.409997,  24.  ],
                 [  8.689162,  49.372064,  23.  ],
                 [  8.670682,  49.396678,   3.  ],
                 [  8.690306,  49.372096,  26.  ],
                 [  8.69358,   49.378182,  13.  ],
                 [  8.63606,   49.395647,   4.  ],
                 [  8.670398,  49.408443,  28.  ],
                 [  8.637578,  49.429881,  25.  ],
                 [  8.682342,  49.403657,   3.  ],
                 [  8.64455,   49.413006,   3.  ],
                 [  8.685483,  49.406815,  16.  ],
                 [  8.691885,  49.372719,  12.  ]])

df_CVRP = pd.DataFrame(CVRP, columns=['longitude', 'latitude', 'demand'])
df_CVRP.columns.name = 'node number'
display(df_CVRP)

node number,longitude,latitude,demand
0,8.683448,49.416961,0.0
1,8.681247,49.412388,22.0
2,8.662789,49.374460,7.0
3,8.663488,49.390504,22.0
4,8.650365,49.424738,8.0
5,8.684574,49.393819,17.0
6,8.636446,49.430018,8.0
7,8.693395,49.401669,25.0
8,8.685109,49.387783,17.0
9,8.670162,49.406377,4.0


In [3]:
api_key = '58d904a497c67e00015b45fc9298e8d961e64b48b066a43e51d39887'
client = openrouteservice.Client(key=api_key)

response = client.distance_matrix(locations=df_CVRP.iloc[:, :2].as_matrix().tolist(), metrics=['duration'])
duration_matrix = np.array(response['durations']).astype(int)
df_duration_matrix = pd.DataFrame(duration_matrix)
display(df_duration_matrix.head())

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0,147,635,383,515,392,561,423,404,248,...,338,617,662,703,250,643,307,487,296,659
1,147,0,523,271,403,280,513,344,292,136,...,226,505,550,590,138,531,195,375,184,547
2,585,473,0,304,699,438,652,510,400,416,...,300,366,496,541,434,781,341,530,377,408
3,412,300,251,0,526,265,609,337,277,243,...,127,439,535,445,261,604,168,388,204,480
4,530,418,755,503,0,512,575,576,524,337,...,458,737,782,609,298,264,427,393,416,779


In [4]:
"""Capacitated Vehicle Routing Problem"""
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

###########################
# Problem Data Definition #
###########################
locations = CVRP[:, :2]
demands = CVRP[:, 2]
num_locations = len(locations)
depot = 0    # The depot is the start and end point of each route.
num_vehicles = 4
capacity = 100


#######################
# Problem Constraints #
#######################

class matrix(object): # pylint: disable=too-few-public-methods
    """Creates callback to return distance between points."""
    def __init__(self, matrix):
        """Initializes the distance matrix."""
        self._matrix = matrix

    def Duration(self, from_node, to_node):
        """Returns the manhattan distance between the two nodes"""
        return self._matrix[from_node][to_node]

class demand(object): # pylint: disable=too-few-public-methods
    """Creates callback to get demands at each location."""
    def __init__(self, demands):
        """Initializes the demand array."""
        self._demands = demands

    def Demand(self, from_node, to_node):
        """Returns the demand of the current node"""
        del to_node
        return self._demands[from_node]
    
########
# Main #
########

"""Entry point of the program"""


# Create Routing Model
routing = pywrapcp.RoutingModel(num_locations, num_vehicles, depot)

# Define weight of each edge
duration_evaluator = matrix(duration_matrix).Duration
routing.SetArcCostEvaluatorOfAllVehicles(duration_evaluator)

# Add Capacity constraint
demand_evaluator = demand(demands).Demand
routing.AddDimension(
    demand_evaluator,
    0, # null capacity slack
    capacity, # vehicle maximum capacity
    True, # start cumul to zero
    "Capacity") # type of constraint

# Setting first solution heuristic (cheapest addition).
search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem
assignment = routing.SolveWithParameters(search_parameters)

# Printing result
total_dist = 0
for vehicle_id in range(num_vehicles):
    index = routing.Start(vehicle_id)
    plan_output = 'Route for vehicle {0}:\n'.format(vehicle_id)
    route_dist = 0
    route_load = 0
    while not routing.IsEnd(index):
        node_index = routing.IndexToNode(index)
        next_node_index = routing.IndexToNode(
            assignment.Value(routing.NextVar(index)))
        route_dist += matrix(duration_matrix).Duration(node_index, next_node_index)
        route_load += demands[node_index]
        plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
        index = assignment.Value(routing.NextVar(index))

    node_index = routing.IndexToNode(index)
    total_dist += route_dist
    plan_output += ' {0} Load({1})\n'.format(node_index, route_load)
    plan_output += 'Distance of the route: {0}m\n'.format(route_dist)
    plan_output += 'Load of the route: {0}\n'.format(route_load)
    print(plan_output)
print('Total Distance of all routes: {0}m'.format(total_dist))


Route for vehicle 0:
 0 Load(0.0) ->  6 Load(8.0) ->  19 Load(11.0) ->  17 Load(36.0) ->  4 Load(44.0) ->  16 Load(72.0) ->  0 Load(72.0)
Distance of the route: 1968m
Load of the route: 72.0

Route for vehicle 1:
 0 Load(0.0) ->  7 Load(25.0) ->  14 Load(38.0) ->  21 Load(50.0) ->  2 Load(57.0) ->  15 Load(61.0) ->  10 Load(85.0) ->  0 Load(85.0)
Distance of the route: 3112m
Load of the route: 85.0

Route for vehicle 2:
 0 Load(0.0) ->  8 Load(17.0) ->  13 Load(43.0) ->  11 Load(66.0) ->  5 Load(83.0) ->  0 Load(83.0)
Distance of the route: 1337m
Load of the route: 83.0

Route for vehicle 3:
 0 Load(0.0) ->  9 Load(4.0) ->  12 Load(7.0) ->  3 Load(29.0) ->  18 Load(32.0) ->  20 Load(48.0) ->  1 Load(70.0) ->  0 Load(70.0)
Distance of the route: 948m
Load of the route: 70.0

Total Distance of all routes: 7365m
